# Template Pipeline
Normalisation&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**-**  
Imputation 1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Median**  
Outlier Detection&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Z Score**  
Imputation 2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Median**  
Feature Selection&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Implicit to model**  
Model&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Lasso Regression**


## Imports

In [2]:
# General
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xgboost as xgb

# ML
from sklearn.linear_model import LinearRegression, Lasso, BayesianRidge, LassoCV, LassoLarsCV
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline


# Custom
import sys,os
sys.path.append('/home/jovyan/work/AML/Task_1/Solution') # I would like a cleaner solution but works for now
import Components.Imputation as Imputation
import Components.Outlier_Detection_1D as Outlier_Detection_1D
import Components.Outlier_Detection_ND as Outlier_Detection_ND
import Components.Feature_Selection as Feature_Selection
import Components.Normalisation as Normalisation
import Components.data_fetching as data_fetching

# CAREFUL:
# If you make changes to a custom module, you have to reload it, i.e rerun this cell
import importlib
importlib.reload(Imputation)
importlib.reload(Outlier_Detection_1D)
importlib.reload(Outlier_Detection_ND)
importlib.reload(Feature_Selection)
importlib.reload(Normalisation)
importlib.reload(data_fetching)

<module 'Components.data_fetching' from '/home/jovyan/work/AML/Task_1/Solution/Components/data_fetching.py'>

## Data Cleaning

In [5]:
X, y = data_fetching.get_train_data()
x_test_final = data_fetching.get_test_data()

mv_mask = Imputation.missing_values_mask(X)
X = Imputation.median(X)
X = Normalisation.gaussian(X)
X,y,mv_mask = Outlier_Detection_ND.magic_indices(X,y,n_outliers=50,mask=mv_mask)
X = Outlier_Detection_1D.z_score(X,x_extra=x_test_final)
X = X.mask(mv_mask) # Re-impute the ones we imputed before
X,x_test_final = Imputation.iterative_regression2(X,x_test_final)
x_test_final = data_fetching.get_test_data() # I don't want to impute x_test_final based on X as well... data leak maybe?
x_test_final = Imputation.mean(x_test_final)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)

## Model Training

### Test

In [22]:
# pipe = Pipeline([('feature_selection', SelectFromModel(LassoLarsCV())),
# ('regressor', GradientBoostingRegressor(n_estimators=100))])

gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(x_train, y_train)
predictions = gbm.predict(test_X)

print('Train Score:')
y_pred_train = gbm.predict(x_train)
train_score = r2_score(y_train, y_pred_train)
print(train_score)

print('Test Score')
y_pred_test = gbm.predict(x_test)
test_score = r2_score(y_test, y_pred_test)
print(test_score)

In [23]:
scores = cross_val_score(pipe, x_train, np.ravel(y_train), cv=5, scoring='r2')
print(scores)
print(np.mean(scores))

[0.56811479 0.51315291 0.55132365 0.52312509 0.60499194]
0.5521416777686738


## Final Prediction

In [4]:
pipe.fit(X, np.ravel(y))

x_test_final = data_fetching.get_test_data()
x_test_final = Imputation.iterative_regression2(x_test_final,X)
# Note: They said no outliers were introduced in x_test_final so no need to perform outlier detection
y_pred = pipe.predict(x_test_final,X)
plt.hist(y_pred)

y_pred_pd = pd.DataFrame(data=y_pred, columns=["y"])
y_pred_pd.to_csv('../../Predictions/RFE40_GradientBoosting.csv', index_label='id')


NameError: name 'pipe' is not defined